In [6]:
import numpy as np

In [2]:
import torch #基本モジュール
from torch.autograd import Variable #自動微分用
import torch.nn as nn #ネットワーク構築用
import torch.optim as optim #最適化関数
import torch.nn.functional as F #ネットワーク用の様々な関数
import torch.utils.data #データセット読み込み関連
import torchvision #画像関連
from torchvision import datasets, models, transforms #画像用データセット諸々

In [4]:
x = torch.Tensor(5, 3) #5x3のTensorの定義
y = torch.rand(5, 3) #5x3の乱数で初期化したTensorの定義
z = x + y #普通に演算も可能
z

tensor([[0.6734, 0.7255, 0.8297],
        [0.6392, 0.6711, 0.0583],
        [0.4717, 0.7514, 0.7161],
        [0.1316, 0.7292, 0.0428],
        [0.3596, 0.7041, 0.1677]])

In [7]:
x = np.random.rand(5, 3)
y = torch.from_numpy(x)
z = y.numpy()
z

array([[0.42342585, 0.33555479, 0.29075874],
       [0.36765733, 0.34344352, 0.78701321],
       [0.30387349, 0.78670604, 0.34113698],
       [0.63137639, 0.15118779, 0.57048147],
       [0.93061811, 0.09039157, 0.85976858]])

In [9]:
x = torch.rand(5, 3)
y = Variable(x)
z = torch.pow(y,2) + 2 #y_i**2 + 2
z

tensor([[2.3733, 2.0198, 2.3989],
        [2.0134, 2.0275, 2.2160],
        [2.3059, 2.0322, 2.0132],
        [2.2912, 2.8298, 2.0023],
        [2.1773, 2.6367, 2.0049]])

In [12]:
#画像の変形処理
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#CIFAR-10のTensorに変形前のtrainsetのロード
rawtrainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True)

#CIFAR-10のtrain, testsetのロード
#変形はtransformを適用
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

#DataLoaderの適用->これによりバッチの割り当て・シャッフルをまとめて行うことができる
#batch_sizeでバッチサイズを指定
#num_workersでいくつのコアでデータをロードするか指定(デフォルトはメインのみ)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [13]:
#resnetからのモデルの読み込み
model_ft = models.resnet18(pretrained=True)
model_ft

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to C:\Users\Stand/.cache\torch\checkpoints\resnet18-5c106cde.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
#プレトレーニング済のモデルのフリーズ
for param in model_conv.parameters():
    param.requires_grad = False

# モデルの最終層のみパラメータを書き換える
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

NameError: name 'model_conv' is not defined

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32,500)
        self.fc2 = nn.Linear(500, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x))
        return x

#モデル定義
model = Net()

#Loss関数の指定
criterion = nn.CrossEntropyLoss()

#Optimizerの指定
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

#トレーニング
#エポック数の指定
for epoch in range(2):  # loop over the dataset multiple times

    #データ全てのトータルロス
    running_loss = 0.0 


    for i, data in enumerate(trainloader):

        #入力データ・ラベルに分割
        # get the inputs
        inputs, labels = data

        # Variableに変形
        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # optimizerの初期化
        # zero the parameter gradients
        optimizer.zero_grad()

        #一連の流れ
        # forward + backward + optimize
        print(inputs.shape)
        print(labels.shape)
        outputs = model(inputs)
        print(outputs.shape)

        #ここでラベルデータに対するCross-Entropyがとられる
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # ロスの表示
        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [135]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(700,500)
        self.fc2 = nn.Linear(500, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x))
        return x

#モデル定義
model = Net()

#Loss関数の指定
criterion = nn.CrossEntropyLoss()

#Optimizerの指定
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

inputs = torch.randn(4, 700, requires_grad=False)
labels = torch.empty(4, dtype=torch.long).random_(10)

optimizer.zero_grad()

outputs = model(inputs)
print("input ", (inputs.shape))
print("output ", outputs.shape)
print("label ", labels.shape)
loss = criterion(outputs, labels)
loss.backward()
# print(model.parameters().grad)
print(inputs.grad)

optimizer.step()

# print("%s\n%s" % (outputs.shape, outputs))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


input  torch.Size([4, 700])
output  torch.Size([4, 10])
label  torch.Size([4])
None


In [1]:
loss = nn.CrossEntropyLoss()
input = torch.randn(32, 100, 3, 128, 128, requires_grad=True)
target = torch.empty(32, 3, 128, 128, dtype=torch.long).random_(input.shape[1])
output = loss(input, target)
output.backward()
print(input)
print(input.shape)
print(target)
print(target.shape)
print(output)

NameError: name 'nn' is not defined